In [1]:
import os
%pwd
os.chdir("../")
%pwd

'c:\\Users\\91629\\Desktop\\projects\\SmartPoultryMgt'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    train_data: Path   # <== Added
    valid_data: Path   # <== Added
    test_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        data_ingestion_path = self.config.data_ingestion.unzip_dir

        # Modify paths to point to specific subdirectories
        train_data = Path(os.path.join(data_ingestion_path, "classification_dataset", "train"))
        valid_data = Path(os.path.join(data_ingestion_path, "classification_dataset", "valid"))
        test_data = Path(os.path.join(data_ingestion_path, "classification_dataset", "test"))

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            train_data=train_data,
            valid_data=valid_data,
            test_data=test_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [6]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        """Prepares training and validation data generators."""

        datagenerator_kwargs = dict(rescale=1.0 / 255)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation Generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.valid_data,  # <== CHANGED: Use valid_data
            shuffle=False,
            **dataflow_kwargs
        )

        # Train Generator with Augmentation if enabled
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.train_data,  # <== CHANGED: Use train_data
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        """Trains the model using prepared data generators."""
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-03-18 05:19:29,764: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-18 05:19:29,784: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-18 05:19:29,787: INFO: common: created directory at: artifacts]
[2025-03-18 05:19:29,789: INFO: common: created directory at: artifacts\training]
Found 928 images belonging to 2 classes.
Found 3142 images belonging to 2 classes.
Epoch 1/100
98/98 [==============================] - 67s 537ms/step - loss: 10.4174 - accuracy: 0.6061 - val_loss: 11.4687 - val_accuracy: 0.7263
Epoch 2/100
98/98 [==============================] - 44s 449ms/step - loss: 5.7949 - accuracy: 0.6987 - val_loss: 1.2394 - val_accuracy: 0.8556
Epoch 3/100
98/98 [==============================] - 44s 450ms/step - loss: 3.7200 - accuracy: 0.7463 - val_loss: 1.2720 - val_accuracy: 0.8707
Epoch 4/100
98/98 [==============================] - 43s 436ms/step - loss: 3.7373 - accuracy: 0.7553 - val_loss: 1.2316 - val_accuracy: 0.8750
Epoch 5/1